In [93]:
import geopandas as gpd
import pandas as pd
import requests
import os
import warnings
from fiona.io import ZipMemoryFile
import io
import fiona

In [43]:
dataPath = r'C:\Users\phwh9568\Data\ParcelAtlas'

In [6]:
warnings.filterwarnings("ignore")

In [3]:
countiesPath = r'C:\Users\phwh9568\Data\ParcelAtlas\tl_2022_us_county\tl_2022_us_county.shp'

In [14]:
counties = gpd.read_file(countiesPath, dtype={'STATEFP':str, 'COUNTYFP':str, 'GEOID':str})

In [26]:
counties.sort_values(['GEOID'], inplace=True)


In [27]:
fipsList = counties['GEOID'].to_list()


In [28]:
len(fipsList)

3235

In [47]:
stateFips = pd.read_csv(os.path.join(dataPath,'stateFips.csv'), dtype={'STATEFP':str, 'STATENS':str})

In [70]:
stateFipsDict=dict(zip(stateFips['STATEFP'].to_list(), stateFips['STATE_NAME'].to_list()))

To iterate through buildings data, use this to start:

In [ ]:

for k,v in stateFipsDict.items():
    print(k,v)

In [75]:
stateFipsDict['44']

'Rhode Island'

In [80]:
url = 'https://usbuildingdata.blob.core.windows.net/usbuildings-v2/'

In [86]:
stateUrl = url+stateFipsDict['44'].replace(' ','')+'.geojson.zip'

In [109]:
stateUrl

'https://usbuildingdata.blob.core.windows.net/usbuildings-v2/RhodeIsland.geojson.zip'

In [116]:
del state

In [143]:
stateBuildings = gpd.read_file(requests.get(stateUrl).content)

In [113]:
stateCounties = counties.loc[counties['STATEFP'] == '44']

In [114]:
stateCounties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
2917,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,None,None,None,N,62500773,53359134,+41.7068397,-071.2866874,"POLYGON ((-71.21043 41.68801, -71.21086 41.687..."
3213,44,003,01219778,44003,Kent,Kent County,06,H4,G4020,None,None,None,N,436588773,50686111,+41.6751180,-071.5802819,"POLYGON ((-71.54481 41.60199, -71.54501 41.601..."
2919,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,None,None,None,N,265293779,547001789,+41.5010449,-071.2830626,"POLYGON ((-71.11640 41.48457, -71.11543 41.482..."
1640,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,None,None,None,N,1060552452,67870210,+41.8697678,-071.5786246,"POLYGON ((-71.54735 41.73120, -71.54743 41.731..."
1336,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,None,None,None,N,852827596,604769731,+41.3967920,-071.6202820,"MULTIPOLYGON (((-71.57520 41.32094, -71.57525 ..."


In [140]:
stateFootprints = gpd.sjoin(stateBuildings, stateCounties, how='left', op='within')
stateFootprints.reset_index(inplace=True)
stateFootprints.drop(['index_right'], axis = 1, inplace=True)

In [172]:
nulls = stateFootprints.loc[stateFootprints['GEOID'].isna()]

In [173]:
dropCols = stateCounties.columns.to_list()

drop unwanted columns

In [174]:
dropCols.remove('geometry')

In [175]:
dropCols.append('index')

In [176]:
nulls.drop(dropCols, axis=1, inplace=True)

In [179]:
nulls = gpd.sjoin(nulls, stateCounties, how='left', op='intersects')

In [182]:
nulls

,release,capture_dates_range,geometry,index_right,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,1,,"POLYGON ((-71.11723 41.50012, -71.11726 41.500...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,,"POLYGON ((-71.12216 41.55490, -71.12204 41.554...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,,"POLYGON ((-71.12274 41.55788, -71.12268 41.557...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,,"POLYGON ((-71.12626 41.55723, -71.12628 41.557...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,,"POLYGON ((-71.11785 41.50632, -71.11774 41.506...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392566,1,,"POLYGON ((-71.83946 41.35288, -71.83940 41.352...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392568,1,,"POLYGON ((-71.84033 41.35587, -71.84023 41.355...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392570,1,,"POLYGON ((-71.84110 41.35939, -71.84110 41.359...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392572,1,,"POLYGON ((-71.84172 41.33796, -71.84162 41.337...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
countFootPrints.columns

Index(['index', 'release', 'capture_dates_range', 'geometry', 'STATEFP',
       'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD', 'CLASSFP',
       'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND', 'AWATER',
       'INTPTLAT', 'INTPTLON'],
      dtype='object')

In [121]:
stateGEOIDs = stateCounties['GEOID'].to_list()

In [124]:
cols = stateBuildings.columns

Index(['release', 'capture_dates_range', 'geometry'], dtype='object')

In [125]:
countFootPrints = stateFootprints.loc[stateFootprints['GEOID'] == stateGEOIDs[0]]

In [ ]:
for fips in stateGEOIDs:
    countyFootPrints = stateFootprints.loc[stateFootprints['GEOID'] == fips]
    countyIntersects = nulls.loc[nulls['GEOID'] == fips]
    countyFootPrints = pd.concat([countyFootPrints, countyIntersects], ignore_index=True)
    countyFootPrints.rename(columns={'NAMELSAD':'County', 'index': 'FID'}, inplace = True)
    fileName = fips+'_Buildings.shp'
    countyFootPrints.to_file('Colorado_Microsoft_Footprints\\'+fileName)

In [136]:
len(nulls.loc[nulls['GEOID_right']==stateGEOIDs[0]])

19563

In [135]:
len(countFootPrints)

19542

In [139]:
nulls['GEOID_left']

0           NaN
1           NaN
2           NaN
3         44005
4           NaN
          ...  
392576    44009
392577    44009
392578    44009
392579    44009
392580    44009
Name: GEOID_left, Length: 392616, dtype: object